In [10]:
import numpy as np
import pandas as pd
# from calibration.Calibration import Calibration 


In [11]:
# load rankings
reco_matrix = np.load("test/reco_matrix.npy")[0]
# scores = np.load("reco_matrix_mapped_scores.npy")[0]
print(reco_matrix.shape)
# print(scores.shape)

(943, 10)


In [12]:
import pickle

with open("test/score_dicts.pkl", "rb") as file:
    scores = pickle.load(file)

# print("Loaded List of OrderedDicts:", scores)

In [13]:
df_m = pd.read_csv(
    "./data/ml-100K/u.item",
    sep="|",
    names=[
        "movieID",
        "Name",
        "Date",
        "Video_Date",
        "IMDB_URL",
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ],
    header=None,
    encoding="latin-1",
)
print(df_m.shape)
df_m = df_m[
    [
        "movieID",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ]
]

df_movies_mapped = pd.read_csv(
    "./data/ml-100K/i_id_mapping.csv",
    sep="\t",
    names=["movieID", "itemID"],
    header=None,
    encoding="latin-1",
)
movies = pd.merge(df_m, df_movies_mapped, how="inner", on="movieID")
movies = movies.drop(columns=["movieID"])
movies = movies.sort_values(by="itemID")

(1682, 24)


In [14]:
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
genre = movies[unique_genres]
item_features_numpy = genre.to_numpy()

users = pd.read_csv("./data/ml-100k/u_id_mapping.csv", sep="\t")

users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users

,Gender,userID
0,0,0
1,1,1
2,0,2
3,0,3
4,0,4
...,...,...
938,1,938
939,0,939
940,1,940
941,1,941


In [15]:
def create_genre_column(r):
    all_genres = [g for g in unique_genres if r[g] == 1]
    return "|".join(all_genres)


movies["genres"] = movies.apply(create_genre_column, axis=1)
movies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
240,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Comedy
300,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,Thriller|Mystery|Film-Noir|Crime
375,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Children's|Comedy
50,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,3,Romance|Western|War|Drama
344,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1344,Drama
1192,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1345,Comedy
1176,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1346,Drama
1261,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1347,Drama


In [16]:
item_ids = movies["itemID"].to_numpy()
item_ids

array([   0,    1,    2, ..., 1346, 1347, 1348])

In [17]:
scores.__len__()

943

In [12]:
# config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
# calibration = Calibration(config, movies, 10, unique_genres, users)
# calibration.get_improved_reco(reco_matrix, list(item_ids), scores)

In [93]:
# tahsin kheya
# last modified 11/12/2024
import pandas as pd
import os
import numpy as np
# import torch
# import time
from logging import getLogger
import random
import numpy as np
from scipy.stats import entropy
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
rank = 0
# rank = comm.Get_rank()

class Calibration(object):
    def __init__(self, config, movies, top_k, unique_genres, users):
        # self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        # self.device = torch.device(self.device)
        self.reco_distribution = []
        self.kl = []
        self.top_k = top_k
        self.gkl = []
        self.gender_df = users
        self.actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
        self.actual_genre_dist = self.actual_genre_dist.sort_values(by="userID")
        self.unique_genres = unique_genres
        self.config = config
        self.item_df = movies
        self.actual_distribution_without_id = self.actual_genre_dist.drop(
            columns=["userID"]
        )
        self.actual_distribution_without_id = self.actual_distribution_without_id.drop(
            columns=["user_timestamp_sum"]
        ).to_numpy()
        self.actual_distribution_gender = []

    def get_all_user_recommended_genre_dist(self, topk_reco):
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(np.arange(topk_reco.shape[0]), self.top_k),
                "itemID": topk_reco.flatten(),
                "rank": np.tile(np.arange(1, self.top_k + 1), topk_reco.shape[0]),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")
        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        summed_genre = (
            merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        )
        weight_total = merged_df.groupby("userID")["weight_factor"].sum()
        summed_genre["user_weight_factor"] = summed_genre["userID"].map(weight_total)
        summed_genre = (
            merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        )
        summed_genre = summed_genre[self.unique_genres].div(
            summed_genre["user_weight_factor"], axis=0
        )

        return summed_genre

    def get_recom_distribution(self, reco, uid):
        reco = np.array(reco)
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(uid, len(reco)),
                "itemID": reco.flatten(),
                "rank": np.tile(np.arange(1, len(reco) + 1), 1),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")

        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        summed_genre = (
            merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        )

        weight_total = merged_df.groupby("userID")["weight_factor"].sum()
        summed_genre["user_weight_factor"] = summed_genre["userID"].map(weight_total)

        summed_genre = summed_genre[self.unique_genres].div(
            summed_genre["user_weight_factor"], axis=0
        )

        return summed_genre[self.unique_genres].to_numpy()

    def get_kl_div(self, q_dist, p_dist, a, uid):
        qg_u = (1 - a) * q_dist + a * p_dist
        kl_div = entropy(p_dist, qg_u, base=10)

        return kl_div

    def compute_diversity_score(self, reco_items, uid, l, scores, b):
        alpha = 0.01
        sum_score = 0

        # -----------------the calibration term----------------------------------

        reco_dist = self.get_recom_distribution(reco_items, uid)[0]
        kl = self.get_kl_div(
            reco_dist, self.actual_distribution_without_id[uid], alpha, uid
        )

        # -----------------the calibration term----------------------------------

        # -----------------the fairness term----------------------------------
        # recommended dist mean for each gender
        male_user_ids = self.gender_df[self.gender_df["Gender"] == 0]["userID"]
        male_user_ids = male_user_ids.to_list()
        gender_genre_dist = self.actual_distribution_gender
        if uid in male_user_ids:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[
                1
            ]  # this is the female avg genre pref
        else:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[0]

        gender_kl = self.get_kl_div(reco_dist, compare_dist, alpha, uid)

        # -----------------the fairness term----------------------------------

        for r in range(len(reco_items)):
            sum_score += scores[reco_items[r]]

        return (1 - b) * sum_score - b * gender_kl

    def get_improved_reco(self, top_items, items, scores):
        return self.get_new_recommendations(
            reco=top_items, scores=scores, all_items=items
        )

    def get_gender_genre_dist(self):
        actual_dist_gender = pd.merge(
            self.actual_genre_dist, self.gender_df, on="userID"
        )
        gender_genre_weights_a = actual_dist_gender.groupby("Gender")[
            self.unique_genres
        ].mean()
        self.actual_distribution_gender = gender_genre_weights_a.sort_index()

    def get_new_recommendations(self, reco, scores, all_items):
        """reco is 6040x50 and scores is 6040x3416"""
        self.get_gender_genre_dist()
        b = 0.69  # beta for the fairness term
        l = 0  # lambda for the calibration term
        all_users = []
        top_k = 5
        num_users = len(scores)
        # for u in range(rank*2,rank*2+2):
        # upper_bound= min(num_users,rank*50+50)
        upper_bound=1
        for u in range(rank*50,upper_bound):
            remaining_items = list(range(5))
            # remaining_items = list
            u_calibrated = []
            for k in range(top_k):
                diversity_scores = [
                    self.compute_diversity_score(u_calibrated + [i], u, l, scores[u], b)
                    for i in remaining_items
                ]
                max_index = np.argmax(diversity_scores)

                best_item = remaining_items[max_index]
                u_calibrated.append(best_item)
                remaining_items.pop(max_index)
                print(diversity_scores)
            print(f"user {u} u_calibrate {u_calibrated}")

            all_users.append(u_calibrated)

        return np.array(all_users)


In [94]:
config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
# movies = movie df
calibration = Calibration(config, movies, 10, unique_genres, users)
print(calibration.get_improved_reco(reco_matrix, list(item_ids), scores))

[0.0009277980600468361, 0.288995190213688, -0.31032321437069843, 0.2647069159894122, 0.04134318556337746]
[1.598327527940449, 1.2808291513864225, 1.814387612444905, 1.561571128632902]
[3.069057573503584, 2.747993292668461, 2.7790005479964464]
[3.782256273501914, 4.036060572348273]
[4.760064106624655]
user 0 u_calibrate [1, 3, 0, 4, 2]
[[1 3 0 4 2]]


In [49]:
# tahsin kheya
# last modified 11/12/2024
import pandas as pd
import os
import numpy as np
# import torch
# import time
from logging import getLogger
import random
import numpy as np
from scipy.stats import entropy
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
rank = 0
# rank = comm.Get_rank()

class CalibrationGreddy(object):
    def __init__(self, config, movies, top_k, unique_genres, users):
        # self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        # self.device = torch.device(self.device)
        self.reco_distribution = []
        self.kl = []
        self.top_k = top_k
        self.gkl = []
        self.gender_df = users
        self.actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
        self.actual_genre_dist = self.actual_genre_dist.sort_values(by="userID")
        self.unique_genres = unique_genres
        self.config = config
        self.item_df = movies
        self.actual_distribution_without_id = self.actual_genre_dist.drop(
            columns=["userID"]
        )
        self.actual_distribution_without_id = self.actual_distribution_without_id.drop(
            columns=["user_timestamp_sum"]
        ).to_numpy()
        self.actual_distribution_gender = []

   

    def get_recom_distribution(self, reco, uid, compare_dist,alpha):
        reco = np.array(reco)
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(uid, len(reco)),
                "itemID": reco.flatten(),
                "rank": np.tile(np.arange(1, len(reco) + 1), 1),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")

        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
       
        
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        
       
        # print(f"compare dist {compare_dist} merged_df {merged_df}")
        
        for i, genre in enumerate(self.unique_genres):
            merged_df[genre] = (1 - alpha) * merged_df[genre] + alpha * compare_dist[i]
        
        
        summed_genre = (
            merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        )
        

        return summed_genre[self.unique_genres].to_numpy()


    def compute_diversity_score(self, reco_items, uid, l, scores, b):
        alpha = 0.01
        sum_score = 0
        
        # recommended dist mean for each gender
        male_user_ids = self.gender_df[self.gender_df["Gender"] == 0]["userID"]
        male_user_ids = male_user_ids.to_list()
        gender_genre_dist = self.actual_distribution_gender
        
        if uid in male_user_ids:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[
                1
            ]  # this is the female avg genre pref
        else:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[0]
       

        reco_dist = self.get_recom_distribution(reco_items, uid, compare_dist, alpha)[0] # sum wr(i)q˜(д|i),
        if np.any(reco_dist)==False: #just an extra check here. the only there will be a 0 in here is when the movie has no genres.
            return -9999
        else:
            reco_dist =np.log(reco_dist) # log sum wr(i)q˜(д|i),
            faireness_term = np.sum(compare_dist*reco_dist) 

        for r in range(len(reco_items)):
            sum_score += scores[reco_items[r]]
            
        # print(f"sum score {sum_score} and fair {faireness_term}")
        # print(f"sum score {(1 - b)*sum_score} and fair {b*faireness_term}")

        return (1 - b) * sum_score  +b * faireness_term

    def get_improved_reco(self, top_items, items, scores):
        return self.get_new_recommendations(
            reco=top_items, scores=scores, all_items=items
        )

    def get_gender_genre_dist(self):
        actual_dist_gender = pd.merge(
            self.actual_genre_dist, self.gender_df, on="userID"
        )
        gender_genre_weights_a = actual_dist_gender.groupby("Gender")[
            self.unique_genres
        ].mean()
        self.actual_distribution_gender = gender_genre_weights_a.sort_index()

    def get_new_recommendations(self, reco, scores, all_items):
        """reco is 6040x50 and scores is 6040x3416"""
        self.get_gender_genre_dist()
        b = 0.69  # beta for the fairness term
        l = 0  # lambda for the calibration term
        all_users = []
        top_k = 10
        num_users = len(scores)
        # for u in range(rank*2,rank*2+2):
        upper_bound= min(num_users,rank*50+50)
        upper_bound=1
        for u in range(rank*50,upper_bound):
            # remaining_items = [793,1,2,3,5]
            remaining_items = all_items
            u_calibrated = []
            for k in range(top_k):
                diversity_scores = [
                    self.compute_diversity_score(u_calibrated + [i], u, l, scores[u], b)
                    for i in remaining_items
                ]
                max_index = np.argmax(diversity_scores)
                print(diversity_scores)

                best_item = remaining_items[max_index]
                u_calibrated.append(best_item)
                remaining_items.pop(max_index)
                print(u_calibrated)
            print(f"user {u} u_calibrate {u_calibrated}")

            all_users.append(u_calibrated)

        return np.array(all_users)


In [50]:
config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
# movies = movie df
calibration = CalibrationGreddy(config, movies, 10, unique_genres, users)
print(calibration.get_improved_reco(reco_matrix, list(item_ids), scores))

[-2.947447282992127, -2.722771128394111, -3.2232693921414244, -2.3409831461989876, -2.7213452503442697, -3.1036509483616244, -2.8116799063527615, -2.999994302983458, -2.6165170724502795, -2.6098478606746154, -2.3768538018061847, -3.1033486125094916, -2.8400815546284655, -3.148444134913209, -2.5857145030477655, -2.666362081954772, -2.593996582369561, -2.585432595203216, -3.0818066962274875, -3.9784588023226615, -2.471564548353906, -2.5099164809682915, -2.706564707263711, -2.6954615628085135, -2.777381807763356, -3.065618171199563, -2.66918379817986, -3.474877169899888, -2.5941354588077026, -3.1195924570010414, -2.2156689891199077, -2.0292980320082172, -2.488961723189111, -2.5078514422872615, -2.6942615630506546, -3.6374188249334196, -2.803627064994272, -2.562126187274749, -3.7324642845194695, -2.6240188111761165, -3.303619240097947, -2.2818320853338703, -3.2225739926643655, -2.514006775756645, -2.6017100223720373, -2.157382052836796, -2.767074347134, -1.7346706671179632, -2.174008851751

In [38]:
np.any(np.array([0,0,0.2]) == 0)

True

In [40]:
p = np.array([0.1, 0.3, 0.6])
q = np.array([0.3, 0.4, 0.3])
# q˜(д|u) = (1 − α) · q(д|u) + α · p(д|u)
q_t = (1 - 0.01) * q + 0.01 * p
print(q_t)

[0.298 0.399 0.303]


In [34]:
movies[793:794]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,793,


In [35]:
movies[movies["genres"]==""]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,793,


In [34]:
scores[0][1]

4.4314069747924805

In [ ]:
sum(
    [
        0.0,
        0.01923077,
        0.0,
        0.11965812,
        0.0,
        0.02136753,
        0.0,
        0.00641026,
        0.0,
        0.02564101,
        0.5769231,
        0.02777778,
        0.01282051,
        0.0,
        0.0,
        0.0,
        0.01495726,
        0.0,
        0.17521365,
    ]
)

0.99999999

In [ ]:
item_index = movies.index.get_loc(0)
item_index

24

In [ ]:
reco = [1, 23, 4]
reco_2 = np.array(reco)
reco_2.flatten()

array([ 1, 23,  4])

In [ ]:
nonzero_indices = np.where(p_dist != 0)

In [ ]:
nonzero_indices

(array([0, 2]),)

In [80]:
unique_genres

['Action',
 'Thriller',
 'Romance',
 'Western',
 "Children's",
 'Mystery',
 'Fantasy',
 'Film-Noir',
 'Documentary',
 'Comedy',
 'Adventure',
 'Sci-Fi',
 'Horror',
 'Crime',
 'Musical',
 'War',
 'Animation',
 'Drama']

,Gender,userID
0,0,0
1,1,1
2,0,2
3,0,3
4,0,4
...,...,...
938,1,938
939,0,939
940,1,940
941,1,941


In [51]:
movies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
240,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Comedy
300,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,Thriller|Mystery|Film-Noir|Crime
375,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Children's|Comedy
50,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,3,Romance|Western|War|Drama
344,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1344,Drama
1192,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1345,Comedy
1176,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1346,Drama
1261,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1347,Drama
